In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0] 

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course']
)

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [21]:
boost = {'question': 3.0, 'section': 0.5}
results = index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
)

In [11]:
from openai import OpenAI

In [13]:
client = OpenAI()

In [17]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role':'user', 'content':q}]
)

In [18]:
response.choices[0].message.content

"It depends on the specific course and the institution offering it. Many courses have different policies regarding late enrollment. Here are a few steps you can take:\n\n1. **Check the Course Policy**: Look for information on the course website or syllabus regarding late enrollment.\n\n2. **Contact the Instructor or Administration**: Reach out to the course instructor or the administrative office for information about the possibility of enrolling late.\n\n3. **Consider Alternative Options**: If late enrollment isn't allowed, ask if there are similar courses available at a later date or if you can access course materials.\n\n4. **Look for Open Courses**: Some institutions offer open enrollment for certain courses at any time.\n\nMake sure to act quickly, as there may be deadlines for enrolling."

In [27]:
prompt_template = '''
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}
CONTEXT: 
{context}
'''.strip()

In [28]:
context = ''
for doc in results:
    context = context + f"section {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [29]:
print(context)

section General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can 

In [31]:
prompt = prompt_template.format(question=q, context=context).strip()

In [32]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?
CONTEXT: 
section General course-related questions
question: The course has already started. Can I still join it?
answer: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

section General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadl

In [34]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role':'user', 'content':prompt}]
)
response.choices[0].message.content

'Yes, you can still enroll in the course even though it has already started. You won’t be able to submit some of the homeworks, but you can still take part in the course. To receive a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ projects by the deadline.'